<a href="https://colab.research.google.com/github/Tomawock/NLP_Attack/blob/main/Dataset_analysis_divided_grups.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### **Gruppo 1**

> Non male i vari dataset escluso quello it che non si cpisce cosa sono i dati all'interno. Carini quelli di GMB e delle droghe, molto generici e relativamente fatti bene


## **Amazon recensioni**

In [ ]:
rev= pd.read_csv('Reviews.csv')

In [ ]:
print("Numero RIGHE COLONNE\n",rev.shape)
print("Nomi Colonne\n",rev.columns)
print("Tipo Colonne\n",rev.dtypes)

Numero RIGHE COLONNE
 (568454, 10)
Nomi Colonne
 Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')
Tipo Colonne
 Id                         int64
ProductId                 object
UserId                    object
ProfileName               object
HelpfulnessNumerator       int64
HelpfulnessDenominator     int64
Score                      int64
Time                       int64
Summary                   object
Text                      object
dtype: object


In [ ]:
rev.groupby("Text")['Id'].count()
# dato che le righe son divere dalle righe totali vuo,dire che certe review sono doppie e non sono poche, scartyo di circa 174875 circa 30%

Text
! think I just ruined my dish. open this and poured it in. some past stuck to the lid, I thought I hadn't shook enough, almost all the rest was syrupy liquid with a little more, not much, past on the bottom. I tasted what was dripping from the can-- what the hell!!! It was like corn syrup!!<br />Don't buy this stuff!                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [ ]:
rev['Text'] = rev['Text'].astype(str)
#type(rev.Text.iat[0])
rev['Text_lenght']=rev.Text.str.len()
#type(rev.Text_lenght.iat[0])

443.07809742859575


In [ ]:
print('Caratteri per Testo Medi\t',rev.Text_lenght.mean())
print('Approssimazione di Parole per Frase\t',rev.Text_lenght.mean()/4.7)

Caratteri per Testo Medi	 443.07809742859575
Approssimazione di Parole per Frase	 94.27193562310548


## **Annotated GMB Corpus**

> Note 
Sto dataset e un plao inq uanto 

1.   Non e in csv => necessita conversione per fare le cose fast 
2.   Notazione IOB, mai vista pero non e troppo comoda
3.   Potrebbe pero essere divertente fare questo tipo di classifacazione(Named Entity Recognition),perche il compito e penso non banale e gli attachi potrebbero essere implementabili in modo agile data la struttura in ambi i casi di CNN e LSTM o RNN o BiRNN




In [ ]:
gmb=''
with open('GMB_dataset.txt','rb') as f:
    for line in f:
        gmb+=line.decode(errors='ignore')

In [ ]:
print("OUTPUT\n",gmb)

## **Sentipolc**

> capire il codice dato da sito prima del dataset



In [ ]:
#!/usr/bin/env python
# evaluation script for the SENTIPOLC 2016 shared task

verbose=True

from argparse import ArgumentParser

argparser = ArgumentParser(description='')
argparser.add_argument('-r', dest='result_file', action='store', help='CSV file of the run results')
argparser.add_argument('-g', dest='gold_file', action='store', default="sentipolc16_gold2000.csv", help='gold standard annotation CSV file')
args = argparser.parse_args()

# read gold standard and populate the count matrix
gold = dict()
gold_counts =  {'subj':{'0':0,'1':0},
                'opos':{'0':0,'1':0},
                'oneg':{'0':0,'1':0},
                'iro':{'0':0,'1':0},
                'lpos':{'0':0,'1':0},
                'lneg':{'0':0,'1':0}
                }
with open(args.gold_file) as f:
    for line in f:
        if len(line.split(',')) > 6:
            id, subj, opos, oneg, iro, lpos, lneg, top = map(lambda x: x[1:-1], line.rstrip().split(','))
        else:
            id, subj, opos, oneg, iro, top = map(lambda x: x[1:-1], line.rstrip().split(','))
        #id, subj, opos, oneg, iro, lpos, lneg, top = map(lambda x: x[1:-1], line.rstrip().split(','))
        gold[id] = {'subj':subj, 'opos':opos, 'oneg':oneg, 'iro':iro, 'lpos':lpos, 'lneg':lneg}
        gold_counts['subj'][subj]+=1
        gold_counts['opos'][opos]+=1
        gold_counts['oneg'][oneg]+=1
        gold_counts['iro'][iro]+=1

        if len(line.split(',')) > 6:
            gold_counts['lpos'][lpos]+=1
            gold_counts['lneg'][lneg]+=1
        
        
# read result data
result = dict()
with open(args.result_file) as f:
    for line in f:
        if len(line.split(',')) > 6:
            id, subj, opos, oneg, iro, lpos, lneg, top = map(lambda x: x[1:-1], line.rstrip().split(','))
            result[id]= {'subj':subj, 'opos':opos, 'oneg':oneg, 'iro':iro}
        else:
            id, subj, opos, oneg, iro, top = map(lambda x: x[1:-1], line.rstrip().split(','))
            result[id]= {'subj':subj, 'opos':opos, 'oneg':oneg, 'iro':iro}

# evaluation: single classes
for task in ['subj', 'opos', 'oneg', 'iro']:    #add 'lpos' and 'lneg' if you want to measure literal polairty
    # table header
    if verbose: print("\ntask: {}".format(task))
    if verbose: print("prec. 0\trec. 0\tF-sc. 0\tprec. 1\trec. 1\tF-sc. 1\tF-sc.")
    correct =  {'0':0,'1':0}
    assigned = {'0':0,'1':0}
    precision ={'0':0.0,'1':0.0}
    recall =   {'0':0.0,'1':0.0}
    fscore =   {'0':0.0,'1':0.0}
   
    # count the labels
    for id, gold_labels in gold.iteritems():
        if (not id in result) or result[id][task]=='':
            pass
        else:
            assigned[result[id][task]] += 1                    
            if gold_labels[task]==result[id][task]:
                correct[result[id][task]] += 1

    # compute precision, recall and F-score
    for label in ['0','1']:
        try:
            precision[label] = float(correct[label])/float(assigned[label])
            recall[label] = float(correct[label])/float(gold_counts[task][label])
            fscore[label] = (2.0 * precision[label] * recall[label]) / (precision[label] + recall[label])
        except:
            # if a team doesn't participate in a task it gets default 0 F-score
            fscore[label] = 0.0
            
    # write down the table
    print("{0:.4f}\t{1:.4f}\t{2:.4f}\t{3:.4f}\t{4:.4f}\t{5:.4f}\t{6:.4f}".format( 
            precision['0'], recall['0'], fscore['0'], 
            precision['1'], recall['1'], fscore['1'],
            (fscore['0'] + fscore['1'])/2.0)
    )
            
# polarity evaluation needs a further step
if verbose: print ("\ntask: polarity")
if verbose: print ("Combined F-score")
correct =  {'opos':{'0':0,'1':0}, 'oneg':{'0':0,'1':0}}
assigned = {'opos':{'0':0,'1':0}, 'oneg':{'0':0,'1':0}}
precision ={'opos':{'0':0.0,'1':0.0}, 'oneg':{'0':0.0,'1':0.0}}
recall =   {'opos':{'0':0.0,'1':0.0}, 'oneg':{'0':0.0,'1':0.0}}
fscore =   {'opos':{'0':0.0,'1':0.0}, 'oneg':{'0':0.0,'1':0.0}}

# count the labels
for id, gold_labels in gold.iteritems():
    for cl in ['opos','oneg']:
        if (not id in result) or result[id][cl]=='':
            pass
        else:
            assigned[cl][result[id][cl]] += 1
            if gold_labels[cl]==result[id][cl]:
                correct[cl][result[id][cl]] += 1
            
# compute precision, recall and F-score
for cl in ['opos','oneg']:
    for label in ['0','1']:
        try:
            precision[cl][label] = float(correct[cl][label])/float(assigned[cl][label])
            recall[cl][label] = float(correct[cl][label])/float(gold_counts[cl][label])
            fscore[cl][label] = float(2.0 * precision[cl][label] * recall[cl][label]) / float(precision[cl][label] + recall[cl][label])
        except:
            fscore[cl][label] = 0.0

fscore_pos = (fscore['opos']['0'] + fscore['opos']['1'] ) / 2.0
fscore_neg = (fscore['oneg']['0'] + fscore['oneg']['1'] ) / 2.0

# write down the table
print ("{0:.4f}".format((fscore_pos + fscore_neg)/2.0))

usage: ipykernel_launcher.py [-h] [-r RESULT_FILE] [-g GOLD_FILE]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-d4fba6fc-053a-4c1e-ad84-06c1e4beec59.json


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## **UCI ML Drug Review**

> NOTE

1.   Review non sono univoche vedi codice sotto
2.   possibile solo fare classificazione ? Sentiment analisys sul rating ?
3.   Classificazione dalla review al farmaco ?





In [ ]:
rev= pd.read_csv('drugsComTrain_raw.csv')

In [ ]:
print("Numero RIGHE COLONNE\n",rev.shape)
print("Nomi Colonne\n",rev.columns)
print("Tipo Colonne\n",rev.dtypes)

Numero RIGHE COLONNE
 (161297, 7)
Nomi Colonne
 Index(['uniqueID', 'drugName', 'condition', 'review', 'rating', 'date',
       'usefulCount'],
      dtype='object')
Tipo Colonne
 uniqueID        int64
drugName       object
condition      object
review         object
rating          int64
date           object
usefulCount     int64
dtype: object


In [ ]:
rev.groupby("review")['uniqueID'].count()
# dato che le righe son divere dalle righe totali vuo,dire che certe review sono doppie e non sono poche, scartyo di circa 48968 circa 30 %

review
"\r\n\r\n please tell the ones who is suffering from anxiety to use lavender chamomile spray by air wick.  it gives immediate relief , doctors not letting know patients about this. please spread the word!!.  Please keep this post here."                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

### **Gruppo 2**

> Non male i vari dataset escluso quello it che non si cpisce cosa sono i dati all'interno. Carini quelli di GMB e delle droghe, molto generici e relativamente fatti bene

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## **Fake news dataset**

> NOTE
Fatto bene, circa 50% e 50% di vere e di false,Non troppo complesso.



In [ ]:
rev= pd.read_csv('Fake.csv')
rev2= pd.read_csv('True.csv')

In [ ]:
print("Numero RIGHE COLONNE\n",rev.shape)
print("Nomi Colonne\n",rev.columns)
print("Tipo Colonne\n",rev.dtypes)

Numero RIGHE COLONNE
 (3054, 6)
Nomi Colonne
 Index(['sentence', 'id_sentence', 'score', 'polarities', 'aspects_position',
       'aspects'],
      dtype='object')
Tipo Colonne
 sentence            object
id_sentence         object
score                int64
polarities          object
aspects_position    object
aspects             object
dtype: object


In [ ]:
print("Numero RIGHE COLONNE\n",rev2.shape)
print("Nomi Colonne\n",rev2.columns)
print("Tipo Colonne\n",rev2.dtypes)

Numero RIGHE COLONNE
 (21417, 4)
Nomi Colonne
 Index(['title', 'text', 'subject', 'date'], dtype='object')
Tipo Colonne
 title      object
text       object
subject    object
date       object
dtype: object


## **ATE ABISTA**

> NOTE:
Fatto bene piu comprenseibile piu dell'altro gruppo, si possono fare vaire classificazioni:
1.   Aspect Term Extraction (ATE)
2.   Aspect Based Sentiment Analysis (ABSA)
3.   Sentiment Analysis (SA).

Risulta ingoto il significato delle **"polarities"**

gia diviso in Train Test e validation, ma i vari dataset sono overlapping 







In [ ]:
rev= pd.read_json('ate_absita_training.ndjson',lines=True)#formato non standard di json ma facilmente parsabile

In [ ]:
print("Numero RIGHE COLONNE\n",rev.shape)
print("Nomi Colonne\n",rev.columns)
print("Tipo Colonne\n",rev.dtypes)

Numero RIGHE COLONNE
 (3054, 6)
Nomi Colonne
 Index(['sentence', 'id_sentence', 'score', 'polarities', 'aspects_position',
       'aspects'],
      dtype='object')
Tipo Colonne
 sentence            object
id_sentence         object
score                int64
polarities          object
aspects_position    object
aspects             object
dtype: object


In [ ]:
i=913
print(rev.aspects_position[i])
print(rev.aspects[i])
print(rev.polarities[i])
print(rev.score[i])
print(rev.sentence[i])

[[107, 126], [85, 105], [71, 82]]
['non troppo rumoroso', 'tre velocità diverse', 'ventilatore']
[[1, 1], [1, 0], [1, 0]]
5
Consegna puntualissima,  cosa volete che vi dica fa il suo dovere è un ventilatore,  tre velocità diverse, non troppo rumoroso e subito montato            


## **ChemProt**

> NOTE devo leggere il readme del dataset, comunque sembra menata

